In [1]:
import pandas as pd

In [2]:
retail = pd.read_csv('5_data.csv.zip', compression='zip', encoding='ISO-8859-1')

In [7]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France


In [4]:
retail_columns = retail.columns

In [5]:
retail_columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

**Проверьте, встречаются ли в данных повторяющиеся наблюдения, и в качестве ответа укажите их количество. 
Если они есть, то удалите их из retail**

In [8]:
len(retail) - len(retail.drop_duplicates())

5268

In [9]:
retail = retail.drop_duplicates()

**Сколько всего транзакций отменили пользователи? 
Каждый отдельный тип продукта считайте за отдельную транзакцию - уникальные номера считать не надо.**

In [10]:
retail.InvoiceNo.str[0].unique() # str[0] из строки InvoiceNo берет 1-ю букву

array(['5', 'C', 'A'], dtype=object)

In [11]:
retail.loc[retail.InvoiceNo.str[0] == 'C'].shape

(9251, 8)

**Теперь отфильтруйте данные и оставьте в retail только те заказы, где Quantity > 0. 
В качестве ответа укажите число оставшихся строк.**

In [13]:
retail = retail.loc[retail.Quantity > 0]

In [14]:
retail.shape

(526054, 8)

**Посчитайте число заказов для каждого пользователя (CustomerID) из Германии (Germany). 
Оставьте только тех, кто совершил более N транзакций (InvoiceNo), где N – 80% процентиль. 
Запишите полученные id пользователей в germany_top (не весь датафрейм, только id).**

In [12]:
invoice_dist = retail\
    .query('Country == "Germany"')\
    .groupby('CustomerID', as_index=False)\
    .agg(total_invoices=('InvoiceNo', 'nunique')) # создаем отдельную колонку с количеством уникальных транзакций

In [13]:
invoice_dist.head()

,CustomerID,total_invoices
0,12426.0,1
1,12427.0,5
2,12468.0,3
3,12471.0,49
4,12472.0,13


In [14]:
N = invoice_dist.total_invoices.quantile(0.8) # 80% данных, 80 процентиль

In [15]:
germany_top = invoice_dist.loc[invoice_dist.total_invoices > N]['CustomerID'] # находим тех кто сделал больше 80% заказов
                                                                                                            # (больше 8 заказов)

In [16]:
germany_top

3     12471.0
4     12472.0
6     12474.0
8     12476.0
12    12481.0
16    12500.0
37    12569.0
44    12600.0
52    12619.0
53    12621.0
57    12626.0
65    12647.0
70    12662.0
78    12705.0
79    12708.0
80    12709.0
82    12712.0
84    12720.0
Name: CustomerID, dtype: float64

**Используя объект с id пользователей (germany_top), полученный на предыдущем шаге, 
отфильтруйте наблюдения и оставьте в данных записи только по интересующим нас юзерам. 
Результирующий датафрейм запишите в top_retail_germany.**

In [17]:
top_retail_germany = retail.query('CustomerID in @germany_top') # @var позволяет использовать переменную в запросе

In [18]:
top_retail_germany.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
1109,536527,22809,SET OF 6 T-LIGHTS SANTA,6,12/1/2010 13:04,2.95,12662.0,Germany
1110,536527,84347,ROTATING SILVER ANGELS T-LIGHT HLDR,6,12/1/2010 13:04,2.55,12662.0,Germany
1111,536527,84945,MULTI COLOUR SILVER T-LIGHT HOLDER,12,12/1/2010 13:04,0.85,12662.0,Germany
1112,536527,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,12/1/2010 13:04,1.65,12662.0,Germany
1113,536527,22244,3 HOOK HANGER MAGIC GARDEN,12,12/1/2010 13:04,1.95,12662.0,Germany


**Сгруппируйте top_retail_germany по коду товара (StockCode). 
Какой из продуктов добавляли в корзину чаще всего, кроме POST?**

In [19]:
top_retail_germany\
    .groupby('StockCode', as_index=False)\
    .agg({'Quantity': 'count'})\
    .sort_values('Quantity', ascending=False)

,StockCode,Quantity
1167,POST,219
409,22326,62
452,22423,55
411,22328,45
474,22467,37
...,...,...
743,23008,1
739,22999,1
44,20717,1
333,22119,1


**Вернемся к анализу полного датасета retail. Создайте колонку Revenue с суммой покупки, используя колонки Quantity и UnitPrice.**

In [22]:
retail = retail.assign(Revenue = retail.Quantity * retail.UnitPrice)

In [23]:
retail

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,Revenue
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12/9/2011 12:50,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60


**Для каждой транзакции (InvoiceNo), посчитайте финальную сумму заказа. 
В качестве ответа укажите топ-5 (InvoiceNo) по сумме заказа (через запятую с пробелом и в порядке убывания TotalRevenue)**

In [24]:
top_5_Revenue = retail\
    .groupby('InvoiceNo')\
    .agg({'Revenue': 'sum'})\
    .sort_values('Revenue', ascending=False)\
    .rename(columns=({'Revenue': 'TotalRevenue'}))\
    .head(5)

In [25]:
top_5_Revenue

,TotalRevenue
InvoiceNo,
581483,168469.60
541431,77183.60
574941,52940.94
576365,50653.91
556444,38970.00


In [27]:
top_5_Revenue.index

Index(['581483', '541431', '574941', '576365', '556444'], dtype='object', name='InvoiceNo')